In [1]:
from database.market import Market
from database.strategy import Strategy
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm
import pytz
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None

In [2]:
market = Market()
strategy = Strategy()

In [3]:
market.connect()
prices = market.retrieve("prices")
market.disconnect()

In [4]:
prices["date"] = pd.to_datetime(prices["date"])
prices["year"] = [x.year for x in prices["date"]]
prices["quarter"] = [x.quarter for x in prices["date"]]
prices["week"] = [x.week for x in prices["date"]]

In [5]:
hl_sim = []
strategy.connect()
for ticker in tqdm(prices["ticker"].unique()):
    try:
        ticker_sim = strategy.retrieve_hlt_data(ticker)
        complete = ticker_sim.groupby(["year","week","ticker"]).mean().reset_index()
        hl_sim.append(complete)
    except:
        continue
strategy.disconnect()

100%|███████████████████████████████████████████████| 501/501 [02:19<00:00,  3.59it/s]


In [6]:
sim = pd.concat(hl_sim)

In [7]:
final_sim = prices.merge(sim,on=["year","week","ticker"],how="left").dropna()

In [8]:
final_sim["weekly_delta"] = (final_sim["high_level_prediction"] - final_sim["adjClose"]) / final_sim["adjClose"]

In [70]:
trades = []
analysis = []
start = final_sim["date"].min()
end = final_sim["date"].max()
seats = 11
for delta in tqdm(range(0,15,5)):
    d = delta/100
    for score in range(60,100,10):
        req = float(score/100)
        for seat in range(seats):
            date = start
            while date <= end:
                if date.weekday() > 4:
                    date = date + timedelta(days=1)
                else:
                    try:
                        todays_sim = final_sim[(final_sim["date"]==date) 
                                               & (final_sim["weekly_delta"] > d) \
                                              & (final_sim["high_level_score"] >= req)].sort_values("weekly_delta",ascending=False)
                        analysis.append({"d":d,"score":score,"date":date,"recs":todays_sim.index.size})
                        if todays_sim.index.size < 1 or seat > todays_sim.index.size:
                            date = date + timedelta(days=1)
                        else:
                            try:
                                trade = todays_sim.iloc[seat]
                                ticker = trade["ticker"]
                                buy_price = trade["adjClose"]
                                exit_price = buy_price * (1+d)
                                max_date = date + timedelta(days=7)
                                exits = final_sim[(final_sim["ticker"]==ticker) & 
                                                    (final_sim["date"] > date) & \
                                                  (final_sim["date"] <= max_date) & \
                                                  (final_sim["adjClose"] >= exit_price)
                                                ].sort_values("date").iloc[1:]
                                if exits.index.size < 1:
                                    exits = final_sim[(final_sim["ticker"]==ticker) & 
                                                  (final_sim["year"] == date.year) & \
                                                    (final_sim["date"] > max_date)].sort_values("date")
                                    sell_trade = exits.iloc[0]
#                                     exits = final_sim[(final_sim["ticker"]==ticker) & 
#                                                   (final_sim["year"] == date.year) & \
#                                                     (final_sim["date"] > max_date) &
#                                                   (final_sim["adjClose"] >= buy_price)].sort_values("date")
#                                     if exits.index.size > 1:
#                                         sell_trade = exits.iloc[0]
#                                     else:
#                                         exits = final_sim[(final_sim["ticker"]==ticker) & 
#                                                   (final_sim["year"] == date.year)].sort_values("date")
#                                         sell_trade = exits.iloc[exits.index.size -1]
                                else:
                                    sell_trade = exits.iloc[0]
                                trade["sell_date"] = sell_trade["date"]
                                trade["sell_price"] = sell_trade["adjClose"]
                                trade["delta"] = (trade["sell_price"] - trade["adjClose"]) / trade["adjClose"]
                                trade["score"] = score
                                trade["d"] = d
                                trade["seat"] = seat
                                trades.append(trade)
                                date = trade["sell_date"] + timedelta(days=1)
                            except Exception as e:
                                date = date +timedelta(days=1)
                    except Exception as e:
                        date = date + timedelta(days=1)
                        continue

100%|██████████████████████████████████████████████████| 3/3 [14:40<00:00, 293.54s/it]


In [71]:
t = pd.DataFrame(trades)

In [72]:
a = []
seats = 11
for delta in tqdm(range(0,15,5)):
    d = delta/100
    for score in tqdm(range(60,100,10)):
        score_trades = t[(t["score"]==score) & 
                         (t["d"]==d)]
        cash = []
        for seat in range(seats):
            initial = float(100 / seats)
            seat_trades = score_trades[score_trades["seat"]==seat]
            for delta in seat_trades["delta"]:
                initial = initial * (1+delta)
                cash.append(initial)
            analysis = {}
            analysis["score"] = score
            analysis["trades"] = seat_trades.index.size
            if seat_trades.index.size > 0:
                analysis["winrate"] = seat_trades[score_trades["delta"] > 0].index.size / seat_trades.index.size
                seat_trades["hpr"] = (seat_trades["sell_date"] - seat_trades["date"])
                seat_trades["hpr"] =[x.days for x in seat_trades["hpr"]]
                analysis["hpr"] = seat_trades["hpr"].mean()
                analysis["seat"] = seat
                analysis["d"] = d
                analysis["score"] = score
                analysis["pv"] = initial
                a.append(analysis)

  0%|                                                           | 0/4 [00:00<?, ?it/s]C:\Users\ericj\AppData\Local\Temp/ipykernel_13248/1656700949.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  analysis["winrate"] = seat_trades[score_trades["delta"] > 0].index.size / seat_trades.index.size

100%|███████████████████████████████████████████████████| 4/4 [00:00<00:00, 43.95it/s]

100%|███████████████████████████████████████████████████| 3/3 [00:00<00:00, 10.41it/s]


In [73]:
final = pd.DataFrame(a)

In [74]:
final

,score,trades,winrate,hpr,seat,d,pv
0,60,150,0.680000,5.220000,0,0.0,22.157277
1,60,140,0.750000,5.757143,1,0.0,30.417251
2,60,143,0.692308,5.461538,2,0.0,20.252227
3,60,148,0.668919,5.331081,3,0.0,9.530491
4,60,150,0.740000,5.246667,4,0.0,14.782041
...,...,...,...,...,...,...,...
127,90,38,0.526316,8.342105,6,0.1,5.997635
128,90,38,0.605263,8.263158,7,0.1,10.611024
129,90,38,0.500000,8.289474,8,0.1,12.376152
130,90,38,0.500000,8.263158,9,0.1,10.794299


In [75]:
hl = final.pivot_table(index=["score","d"],columns="seat",values="pv")

In [76]:
report = final.merge(hl.reset_index(),on=["score","d"]).drop(["pv","seat"],axis=1).groupby(["score","d"]).mean()
report["pv"] = [sum([row[1][x] for x in range(seats)]) for row in report.iterrows()]

In [77]:
report.sort_values("pv",ascending=False).reset_index()

,score,d,trades,winrate,hpr,0,1,2,3,4,5,6,7,8,9,10,pv
0,60,0.05,104.545455,0.559716,8.075952,15.733150,20.661315,56.610841,19.135205,22.492837,24.335137,21.258703,7.340955,50.593410,14.001299,12.331080,264.493931
1,70,0.05,104.545455,0.559716,8.075952,15.733150,20.661315,56.610841,19.135205,22.492837,24.335137,21.258703,7.340955,50.593410,14.001299,12.331080,264.493931
2,60,0.10,101.272727,0.540820,8.393072,13.490865,17.669832,21.523290,11.557472,22.263921,46.534727,12.911199,11.979677,22.130926,18.001818,16.136077,214.199805
3,70,0.10,101.272727,0.540820,8.393072,13.490865,17.669832,21.523290,11.557472,22.263921,46.534727,12.911199,11.979677,22.130926,18.001818,16.136077,214.199805
4,80,0.05,76.454545,0.556871,8.060595,12.013236,16.483618,33.076441,14.894146,14.828604,21.874233,19.717451,10.022948,30.275326,11.044418,10.903406,195.133829
5,80,0.10,73.727273,0.556334,8.407381,11.197803,11.394393,23.232049,14.320908,13.872583,43.235084,11.390935,16.356412,19.471559,14.989825,14.131084,193.592634
6,60,0.00,147.181818,0.710067,5.335839,22.157277,30.417251,20.252227,9.530491,14.782041,20.611151,19.149115,5.996919,22.866868,14.965727,10.906023,191.635089
7,70,0.00,147.181818,0.710067,5.335839,22.157277,30.417251,20.252227,9.530491,14.782041,20.611151,19.149115,5.996919,22.866868,14.965727,10.906023,191.635089
8,80,0.00,107.090909,0.714334,5.362288,17.099511,14.780427,17.869692,12.135073,11.528631,20.511164,20.755389,9.559996,20.326588,11.283950,7.499682,163.350103
9,90,0.05,39.636364,0.542404,8.473085,6.000943,8.685351,14.703101,13.054053,9.427014,9.335135,5.594187,7.058826,13.188883,10.310855,3.499631,100.857977


In [78]:
score = report.sort_values("pv",ascending=False).reset_index().iloc[0]["score"].item()
d = report.sort_values("pv",ascending=False).reset_index().iloc[0]["d"].item()
benchmark = t[(t["score"]==score) & (t["d"]==d)]

In [79]:
timelines =[]
for seat in tqdm(range(seats)):
    seat_trades = benchmark[benchmark["seat"]==seat]
    initial = float(100/seats)
    timeline = []
    for row in seat_trades.iterrows():
        initial = initial * (1+row[1]["delta"])
        timeline.append(initial)
    seat_trades["pv"] = timeline
    timelines.append(seat_trades)

100%|████████████████████████████████████████████████| 11/11 [00:00<00:00, 224.44it/s]


In [80]:
market.connect()
sp5 = market.retrieve("sp500")
market.disconnect()
sp5.rename(columns={"Symbol":"ticker"},inplace=True)
benchmark = benchmark.merge(sp5,on="ticker",how="left")

In [81]:
sp5.columns

Index(['_id', 'ticker', 'Security', 'SEC filings', 'GICS Sector',
       'GICS Sub-Industry', 'Headquarters Location', 'Date first added', 'CIK',
       'Founded'],
      dtype='object')

In [82]:
# industry check
benchmark.groupby(["year","GICS Sector"]).mean().sort_values("delta",ascending=False).reset_index()

,year,GICS Sector,close,high,low,open,volume,adjClose,adjHigh,adjLow,...,quarter_y,high_level_prediction,high_level_score,weekly_delta,sell_price,delta,score,d,seat,CIK
0,2020,Consumer Discretionary,212.883143,214.998557,209.787823,211.976571,4.507608e+07,212.868864,214.983794,209.773867,...,2.388776,336.943075,0.905722,1.139739,212.609456,0.043713,60.0,0.05,4.114286,4.195997e+05
1,2019,Information Technology,98.127045,99.102273,96.928523,98.150455,1.480371e+06,96.536060,97.494618,95.353699,...,2.250000,132.298065,0.899959,0.434077,98.775602,0.041112,60.0,0.05,6.772727,9.007065e+05
2,2021,Real Estate,50.642353,51.422253,49.898359,50.727647,2.094778e+06,49.602576,50.364419,48.876497,...,1.264706,89.821846,0.776207,0.894801,50.679316,0.028900,60.0,0.05,3.588235,8.308662e+05
3,2021,Consumer Discretionary,112.041000,114.263523,110.939667,112.949833,4.336128e+06,108.817732,110.989240,107.800719,...,1.866667,149.879236,0.770469,0.529085,110.458445,0.021022,60.0,0.05,5.133333,6.851386e+05
4,2021,Energy,24.473800,25.136320,24.149100,24.793160,9.485128e+06,24.302826,24.961045,23.980153,...,2.080000,31.896577,0.758486,0.325734,24.241350,0.018471,60.0,0.05,7.600000,6.183391e+05
5,2019,Industrials,199.789778,202.073798,197.572731,200.100889,2.204199e+06,193.345191,195.546908,191.201167,...,2.541048,272.422802,0.900701,0.411735,195.543476,0.017908,60.0,0.05,6.377778,4.766264e+05
6,2020,Information Technology,82.267037,82.968889,81.367593,82.077778,8.466194e+06,81.775603,82.471606,80.882493,...,2.870370,117.808053,0.901378,0.598041,82.096810,0.017720,60.0,0.05,4.370370,1.356717e+06
7,2020,Financials,105.628000,106.716333,104.546333,105.337333,5.193133e+06,104.201090,105.271607,103.133542,...,2.234249,150.144196,0.919834,0.682689,102.994057,0.016250,60.0,0.05,6.100000,7.939779e+05
8,2020,Industrials,156.476064,158.642234,154.280400,156.713543,1.715148e+07,155.611276,157.766877,153.428723,...,2.127771,217.832015,0.877312,0.553774,154.428988,0.016066,60.0,0.05,6.446809,7.960569e+05
9,2021,Communication Services,67.006296,68.261511,66.246667,67.568889,1.491104e+07,66.791982,68.042305,66.035652,...,1.370370,104.966150,0.774265,0.739649,67.300590,0.015240,60.0,0.05,5.037037,8.694297e+05


In [83]:
benchmark.groupby(["year","ticker"]).mean().sort_values("delta",ascending=False).reset_index().merge(sp5[["ticker","Security"]],on="ticker",how="left").head(20)

,year,ticker,close,high,low,open,volume,adjClose,adjHigh,adjLow,...,high_level_prediction,high_level_score,weekly_delta,sell_price,delta,score,d,seat,CIK,Security
0,2019,ENPH,4.920000,4.930000,4.610000,4.630000,7.564780e+05,4.920000,4.930000,4.610000,...,21.171026,0.859901,3.303054,9.260000,0.882114,60.0,0.05,1.000000,1463101.0,Enphase Energy
1,2020,CZR,9.455000,13.820000,9.127500,11.655000,1.243698e+07,9.455000,13.820000,9.127500,...,23.799507,0.930153,1.528539,15.870000,0.680231,60.0,0.05,6.000000,1590895.0,Caesars Entertainment
2,2020,PENN,6.530000,9.690000,6.125000,9.660000,1.681361e+07,6.530000,9.690000,6.125000,...,19.353352,0.930153,2.273958,10.550000,0.646207,60.0,0.05,4.000000,921738.0,Penn National Gaming
3,2020,NCLH,8.400000,9.900000,8.080000,9.160000,3.318173e+07,8.400000,9.900000,8.080000,...,18.073953,0.862260,1.151661,11.010000,0.310714,60.0,0.05,10.000000,1513761.0,Norwegian Cruise Line Holdings
4,2020,AAL,11.459167,11.813333,11.345000,11.636667,6.652391e+07,11.459167,11.813333,11.345000,...,20.227238,0.930771,0.789463,12.825000,0.129917,60.0,0.05,6.833333,6201.0,American Airlines Group
5,2019,OXY,40.700000,41.120000,40.160000,40.250000,1.048573e+07,38.162491,38.556306,37.656159,...,49.009250,0.883287,0.284226,42.241284,0.106880,60.0,0.05,6.000000,797468.0,Occidental Petroleum
6,2020,HBAN,8.850000,9.233333,8.736667,9.125000,1.105841e+07,8.421571,8.784982,8.315049,...,16.396244,0.940996,1.015759,9.291715,0.105894,60.0,0.05,7.666667,49196.0,Huntington Bancshares
7,2021,NCLH,23.580000,24.915000,23.510000,24.060000,1.541833e+07,23.580000,24.915000,23.510000,...,29.541323,0.722099,0.252813,25.820000,0.094996,60.0,0.05,8.000000,1513761.0,Norwegian Cruise Line Holdings
8,2019,TYL,188.260000,190.620000,187.645000,190.620000,1.537960e+05,188.260000,190.620000,187.645000,...,228.186032,0.918366,0.212079,205.680000,0.092532,60.0,0.05,10.000000,860731.0,Tyler Technologies
9,2021,FANG,68.180000,68.550000,66.120000,67.010000,2.305825e+06,68.180000,68.550000,66.120000,...,88.368065,0.722099,0.296100,74.200000,0.088296,60.0,0.05,9.000000,1539838.0,Diamondback Energy
